In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import numpy as np
import zgoubidoo
from zgoubidoo.commands import *
from zgoubidoo import ureg as _ureg
from zgoubidoo.commands.contrib.iba import *
from zgoubidoo.commands.actions import Fit as _Fit
from zgoubidoo.commands.actions import FitType as _FitType
from zgoubidoo.zgoubi import Zgoubi as _Zgoubi
from zgoubidoo.commands.actions import Fit2 as _Fit2
from zgoubidoo.commands.magnetique import Multipol
from zgoubidoo.commands.contrib.iba import *

In [ ]:
os.environ['ZGOUBI_EXECUTABLE_PATH'] = "/Users/rtesse/packages/zgoubi-build-clion-develop_gnu9/install/bin/"

# Create CGTR line

In [ ]:
cgtr = CGTR(with_fit=True, kinematics=zgoubidoo.Kinematics(140*_ureg.MeV))

#Imposing the fringe field effect :
cgtr.line.SMX.X_E = 0.0 * _ureg.centimeters # 3 times LAM_...
cgtr.line.SMX.X_S = 0.0 * _ureg.centimeters
cgtr.line.SMY.X_E = 0.0 * _ureg.centimeters
cgtr.line.SMY.X_S = 0.0 * _ureg.centimeters


#Modification of the integration path :
cgtr.line.B1G.XPAS = 1.0 * _ureg.centimeters
cgtr.line.B2G.XPAS = 1.0 * _ureg.centimeters
cgtr.line.B3G.XPAS = 3.0 * _ureg.centimeters
cgtr.line.Q1G.XPAS = 1.0 * _ureg.centimeters
cgtr.line.Q2G.XPAS = 1.0 * _ureg.centimeters
cgtr.line.Q3G.XPAS = 1.0 * _ureg.centimeters
cgtr.line.Q4G.XPAS = 1.0 * _ureg.centimeters
cgtr.line.Q5G.XPAS = 1.0 * _ureg.centimeters
cgtr.line.Q6G.XPAS = 1.0 * _ureg.centimeters
cgtr.line.Q7G.XPAS = 1.0 * _ureg.centimeters

cgtr.line.IL = 2
cgtr.line.survey(with_reference_trajectory=True);

# Plot the CGTR with a tracks

In [ ]:
zr = zgoubidoo.Zgoubi()(cgtr.line).collect() #zgoubi results
tracks_global = zr.tracks_global
tracks_frenet = zr.tracks_frenet

### Make a spot @ (0,0)

In [ ]:
tracks = cgtr.spots([[0,0]], with_tracks=True)
zr = zgoubidoo.Zgoubi()(cgtr.line).collect() #zgoubi results

tracks_global_centered = zr.tracks_global
tracks_frenet_centered = zr.tracks_frenet

## Global coordinates

In [ ]:
artist = zgoubidoo.vis.ZgoubidooPlotlyArtist(width=800, height=600)

artist.scatter(
    x=tracks_global['XG'],
    y=tracks_global['YG'],
    line={'width': 2, 'color': 'blue'},
    mode='lines',
    name='original',
    )

artist.scatter(
    x=tracks_global_centered['XG'],
    y=tracks_global_centered['YG'],
    line={'width': 2, 'color': 'red'},
    mode='lines',
    name='centered',
    )

artist.plot_beamline(beamline=cgtr.line, with_frames=True, with_map=True)
artist.render()

## Frenet coordinates

In [ ]:
artist = zgoubidoo.vis.ZgoubidooPlotlyArtist(width=800, height=600)

artist.scatter(
        x=tracks_frenet['SREF'],
        y=tracks_frenet['YT'],
        name='Original',
        line={'width': 2, 'color': 'blue'},
        mode='lines',
    )

artist.scatter(
        x=tracks_frenet_centered['SREF'],
        y=tracks_frenet_centered['YT'],
        line={'width': 2, 'color': 'red'},
        name='Centered',
        mode='lines',
    )

artist.plot_cartouche(beamline=cgtr.line, vertical_position=1.2)
artist.render()

# Scanning on a grid

In [ ]:
sample_x = np.linspace(-5, 5, 11)
sample_y = np.linspace(-5, 5, 11)

spots = np.asarray(list(map(lambda y : list(map(lambda x : [x,y], sample_x)), sample_y))) #convert input as array
spots = spots.reshape(sample_x.size * sample_y.size,2)

tracks = cgtr.spots(spots, with_tracks=True)
fields = np.asarray(list(map(lambda x : tracks.query(f'SPOT_X == {x[0]} and SPOT_Y == {x[1]}')[['SMX__B1', 'SMY__B1']].iloc[0].values, spots)))

## Plot the results

In [ ]:
artist = zgoubidoo.vis.ZgoubidooPlotlyArtist(width=800)
artist.fig['layout']['xaxis']['title'] = "X (cm)"
artist.fig['layout']['yaxis']['title'] = "Y (cm)"
artist.scatter(
        x=tracks.query("LABEL1 == 'ISO'")['Y'] * 100,
        y=tracks.query("LABEL1 == 'ISO'")['Z'] * 100,
        marker={'color': 'blue', 'symbol': 303, 'size': 5},
        name='Nominal',
        mode='markers',
    )
artist.render()

## Plot the field in SMX - SMY

In [ ]:
artist = zgoubidoo.vis.ZgoubidooPlotlyArtist(width=800)
artist.fig['layout']['xaxis']['title'] = "Magnetic field in SMX (T)"
artist.fig['layout']['yaxis']['title'] = "Magnetic field in SMY (T)"
artist.scatter(
        x=fields[:,0],
        y=fields[:,1],
        marker={'color': 'blue', 'symbol': 303, 'size': 5},
        name='Nominal',
        mode='markers',
    )
artist.render()